In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vzhou842/profanity-check/master/profanity_check/data/clean_data.csv')
df.head()

,is_offensive,text
0,0,Then go to the village pump and suggest they c...
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3,0,It is better for Atabay not helping the banned...
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,..."


In [ ]:
df.columns

Index(['is_offensive', 'text'], dtype='object')

In [ ]:
df.shape

(184354, 2)

In [ ]:
df = df[['text', 'is_offensive']]

In [ ]:
df = df.rename(columns = {'is_offensive' : 'Is_Toxic'})

In [ ]:
df.head()

,text,Is_Toxic
0,Then go to the village pump and suggest they c...,0
1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...,1
2,Dis hoe wasnt dis violent on Lottery Ticket 😂😂,1
3,It is better for Atabay not helping the banned...,0
4,"""is in CamelCase. """"SiCKO"""" is not CamelCase,...",0


In [ ]:
n = 25588
print(df.iloc[n]['text'])
print(df[df.columns[1:]].iloc[n])

Glad to hear it ;) Mostly it's been little stuff. You were running the citation bot, not reflinks; they do different things. There are bot-prep tricks that work for each. For example, using the best cite template; change cit web to cite journal and the bot might add volume and issue numbers; issns, and such, too. Cheers,
Is_Toxic    0
Name: 25588, dtype: int64


In [ ]:
n = 2359
print(df.iloc[n]['text'])
print(df[df.columns[1:]].iloc[n])

Hey , long time no see! )  How is Texas?  Is this some sock puppet, or did you forget to log in? )  Yeah, I will forever be someone with an open mind and not tied to a black-and-white attitude.  That will never change. )
Is_Toxic    0
Name: 2359, dtype: int64


In [ ]:
n = 2485
print(df.iloc[n]['text'])
print(df[df.columns[1:]].iloc[n])

"
I dont think ""Hinduism"" as such has taken that position. Hinduism is a compendium of traditions and practices. The most one can say is that in certain Vaishnavite traditions he has been absorbed - as Buddha was before - into the system of incanations of Vishnu/Krishna.   "
Is_Toxic    0
Name: 2485, dtype: int64


In [ ]:
df.count()

text        184350
Is_Toxic    184354
dtype: int64

In [ ]:
df.isnull().sum()

text        4
Is_Toxic    0
dtype: int64

In [ ]:
df  =df.dropna()

In [ ]:
df.isnull().sum()

text        0
Is_Toxic    0
dtype: int64

In [ ]:
df[df[df.columns]== 1].sum()

text              0
Is_Toxic    36845.0
dtype: object

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df['text']
y = df[df.columns[1:]].values

In [ ]:
X.head()

0    Then go to the village pump and suggest they c...
1    ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2       Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3    It is better for Atabay not helping the banned...
4    "is in CamelCase.  ""SiCKO"" is not CamelCase,...
Name: text, dtype: object

In [ ]:
y

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
MAX_FEATURES = 100000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_mode='int', )

In [ ]:
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(16000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
train = dataset.take(int(len(dataset)*.7))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.2))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()

model.add(Embedding(MAX_FEATURES+1, 128))
model.add(Bidirectional(LSTM(64,activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         12800128  
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                        

In [ ]:
history = model.fit(train,epochs=1)

8065/8065 [==============================] - 682s 83ms/step - loss: 0.1398 - accuracy: 0.9507


In [ ]:
input_text = vectorizer('your mother is so stupid she studied for covid test')
res = model.predict(np.array([input_text]))
res

1/1 [==============================] - 0s 20ms/step


array([[0.9993581]], dtype=float32)

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[1:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text
interface = gr.Interface(fn=score_comment, title = "TOXICITY PREDICTOR",
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')
interface.launch(share=True)

In [ ]:
model.save('Toxic_text_classifier.keras')